In [1]:
cd '../'

/scratch/ssd001/home/ama/workspace/ama-at-vector/freq-robust


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import trange
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import grad

from src.attacks import pgd_rand
from src.utils_general import seed_everything
from src.context import ctx_noparamgrad_and_eval
from src.utils_freq import rgb2gray, dct, dct2, idct, idct2, batch_dct2, getDCTmatrix

from models import LR_model
from src.linear_analysis import train_LR
from src.linear_analysis import plot_loss_LR, plot_w_tilde_LR, plot_dw_tilde_LR, plot_loss_adv_LR
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from collections import defaultdict
import ipdb
import copy

# hyper params initalization
_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
args = {"case": 1,
        "itr": 1001,
        "bsize": 128,
        "mu": 1,
        "std": 0.5,
        "lambbda": 0.2,
        "d": 10,
        "lr": 1e-1,
        "eps": 0.1}
model = LR_model(args["d"]).to(_device)

opt = optim.SGD(model.parameters(), lr = args["lr"])
log = train_LR(args, model, opt, _device)



In [5]:
m = 3* torch.ones(5,5) + 2*torch.eye(5)
# m = 3 * torch.ones(5,5)
print(m)
result = torch.eig(m, eigenvectors=True)
print(result)
# print(result[1][:,3])
# print(result[1][:,4].t())
# print(torch.mm(result[1][:,3].view(1,5),result[1][:,3].view(5,1)))
# print(torch.norm(result[1][:,4]))

tensor([[5., 3., 3., 3., 3.],
        [3., 5., 3., 3., 3.],
        [3., 3., 5., 3., 3.],
        [3., 3., 3., 5., 3.],
        [3., 3., 3., 3., 5.]])
torch.return_types.eig(
eigenvalues=tensor([[ 2.0000,  0.0000],
        [17.0000,  0.0000],
        [ 2.0000,  0.0000],
        [ 2.0000,  0.0000],
        [ 2.0000,  0.0000]]),
eigenvectors=tensor([[-0.8944,  0.4472, -0.0256, -0.0289, -0.0237],
        [ 0.2236,  0.4472,  0.8721,  0.2957,  0.1411],
        [ 0.2236,  0.4472, -0.2821,  0.6804, -0.3997],
        [ 0.2236,  0.4472, -0.2821, -0.4737, -0.4833],
        [ 0.2236,  0.4472, -0.2821, -0.4737,  0.7656]]))


In [14]:
Q = result[1]
lam = result[0][:,0]
# print(lam)
recon = torch.mm(torch.mm(Q,torch.diag(lam)),torch.inverse(Q))
print(recon)

tensor([[5.0000, 3.0000, 3.0000, 3.0000, 3.0000],
        [3.0000, 5.0000, 3.0000, 3.0000, 3.0000],
        [3.0000, 3.0000, 5.0000, 3.0000, 3.0000],
        [3.0000, 3.0000, 3.0000, 5.0000, 3.0000],
        [3.0000, 3.0000, 3.0000, 3.0000, 5.0000]])


In [28]:
Q = torch.tensor([[1.,1.,1.,1.,1.],[1.,-1.,0,0,0],[1.,0,-1.,0,0],[1.,0,0,-1.,0],[1.,0,0,0,-1.]])
print(Q)
lam = torch.tensor([17.,2.,2.,2.,2.])
# print(lam)
recon = torch.mm(torch.mm(Q,torch.diag(lam)),torch.inverse(Q))
print(recon)

tensor([[ 1.,  1.,  1.,  1.,  1.],
        [ 1., -1.,  0.,  0.,  0.],
        [ 1.,  0., -1.,  0.,  0.],
        [ 1.,  0.,  0., -1.,  0.],
        [ 1.,  0.,  0.,  0., -1.]])
tensor([[5.0000, 3.0000, 3.0000, 3.0000, 3.0000],
        [3.0000, 5.0000, 3.0000, 3.0000, 3.0000],
        [3.0000, 3.0000, 5.0000, 3.0000, 3.0000],
        [3.0000, 3.0000, 3.0000, 5.0000, 3.0000],
        [3.0000, 3.0000, 3.0000, 3.0000, 5.0000]])


In [13]:
np.ones((5,5))

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [44]:
# m = 3 * np.ones((5,5)) + 2*np.eye(5)
m = 3 * np.ones((5,5))
print(m)
result = np.linalg.eig(m)
print(result)

[[3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]
 [3. 3. 3. 3. 3.]]
(array([ 0., 15.,  0.,  0.,  0.]), array([[-8.94427191e-01,  4.47213595e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 2.23606798e-01,  4.47213595e-01,  8.66025404e-01,
        -4.16333634e-17, -4.16333634e-17],
       [ 2.23606798e-01,  4.47213595e-01, -2.88675135e-01,
        -5.77350269e-01, -5.77350269e-01],
       [ 2.23606798e-01,  4.47213595e-01, -2.88675135e-01,
         7.88675135e-01, -2.11324865e-01],
       [ 2.23606798e-01,  4.47213595e-01, -2.88675135e-01,
        -2.11324865e-01,  7.88675135e-01]]))


In [15]:
m = torch.ones(5,5)
print(m)
result = torch.eig(m, eigenvectors=True)
print(result)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
torch.return_types.eig(
eigenvalues=tensor([[ 0.0000e+00,  0.0000e+00],
        [ 5.0000e+00,  0.0000e+00],
        [ 4.2352e-22,  0.0000e+00],
        [-1.4211e-14,  0.0000e+00],
        [ 1.9478e-43,  0.0000e+00]]),
eigenvectors=tensor([[-0.8944, -0.4472, -0.8944,  0.3338, -0.8944],
        [ 0.2236, -0.4472,  0.2236, -0.8869,  0.2236],
        [ 0.2236, -0.4472,  0.2236,  0.1844,  0.2236],
        [ 0.2236, -0.4472,  0.2236,  0.1844,  0.2236],
        [ 0.2236, -0.4472,  0.2236,  0.1844,  0.2236]]))


In [16]:
m = np.ones([5,5])
print(m)
result = np.linalg.eig(m)
print(result[0])
print(result[1][:,0])

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[5.00000000e+00 5.65333918e-49 0.00000000e+00 0.00000000e+00
 1.73014109e-64]
[-0.4472136 -0.4472136 -0.4472136 -0.4472136 -0.4472136]


In [ ]:
Q = torch.tensor()